In [1]:
import os
import glob
import warnings
warnings.simplefilter("ignore", RuntimeWarning) # Ignore invalid arcsin() in EDD calculation

import dask
import numpy as np
import pandas as pd
import xarray as xr

### Preliminaries

In [2]:
###############################
# Set paths
# UPDATE THIS FOR REPRODUCTION
###############################
nex_path = "/gpfs/group/kaf26/default/public/NEX-GDDP-CMIP6/models"  # location of NEX-GDDP-CMIP6
okr_path = "/gpfs/group/kaf26/default/public/OakRidgeCMIP6"  # location of OakRidge
loca_path = "/gpfs/group/kaf26/default/public/LOCA2"  # location of LOCA

wbm_path = "/gpfs/group/kaf26/default/dcl5300/wbm_soilM_crop_uc_lafferty-etal-2024-tbd_DATA/wbm/wbm_spool/flowdirection206_us/" # for spooled climate data

out_path = "/gpfs/group/kaf26/default/dcl5300/wbm_soilM_crop_uc_lafferty-etal-2024-tbd_DATA/climate/"

In [3]:
# Read all climate drivers 
df_climate_drivers = pd.read_csv("../utils/climate_drivers.csv")

In [14]:
############
# Dask
############
from dask_jobqueue import PBSCluster

cluster = PBSCluster(
    cores=1,
    memory="40GB",
    resource_spec="pmem=40GB",
    # account='open',
    worker_extra_args=["#PBS -l feature=rhel7"],
    walltime="00:30:00",
)

cluster.scale(jobs=10)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.240:33445,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# EDD calculations

In [4]:
# Degree day calculations
def above_threshold_each(mins, maxs, threshold):
    """Use a sinusoidal approximation to estimate the number of Growing
    Degree-Days above a given threshold, using daily minimum and maximum
    temperatures.
    mins and maxs are numpy arrays; threshold is in the same units."""
    
    """
    Code from James Rising (https://github.com/jrising/research-common/blob/master/python/gdd.py)
    """

    # Determine crossing points, as a fraction of the day
    plus_over_2 = (mins + maxs)/2
    minus_over_2 = (maxs - mins)/2
    two_pi = 2*np.pi
    # d0s is the times of crossing above; d1s is when cross below
    d0s = np.arcsin((threshold - plus_over_2) / minus_over_2) / two_pi
    d1s = .5 - d0s

    # If always above or below threshold, set crossings accordingly
    aboves = mins >= threshold
    belows = maxs <= threshold

    d0s[aboves] = 0
    d1s[aboves] = 1
    d0s[belows] = 0
    d1s[belows] = 0

    # Calculate integral
    F1s = -minus_over_2 * np.cos(2*np.pi*d1s) / two_pi + plus_over_2 * d1s
    F0s = -minus_over_2 * np.cos(2*np.pi*d0s) / two_pi + plus_over_2 * d0s
    return F1s - F0s - threshold * (d1s - d0s)

def edd_ufunc(tasmin, tasmax, threshold):
    return xr.apply_ufunc(above_threshold_each,
                          tasmin, tasmax, threshold)

## Spooled climate drivers

In [16]:
############################################
# EDD function for spooled climate drivers
############################################
def edd_spooled(wbm_path, ensemble, model, member, ssp, method, year_start, year_end, thresh, out_path):
    ds_edd_out = []
    # Loop through time slices
    for year in range(year_start, year_end+1):
        try:
            # Read
            if ensemble == "OakRidge":
                tasmax, tasmin = "tmax", "tmin"
                ds_tasmax = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{method}_{ssp}_tmax_daily/{year}.nc")
                ds_tasmin = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{method}_{ssp}_tmin_daily/{year}.nc")
            else:
                tasmax, tasmin = "tasmax", "tasmin"
                ds_tasmax = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{ssp}_tasmax_daily/{year}.nc")
                ds_tasmin = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{ssp}_tasmin_daily/{year}.nc")
            
            # Calculate EDD
            if ensemble == "OakRidge":
                thresh_units = thresh
            else:
                thresh_units = thresh + 273.15
            
            ds_edd = edd_ufunc(tasmin = ds_tasmin[tasmin],
                               tasmax = ds_tasmax[tasmax],
                               threshold = thresh_units)
            # Sum to monthly
            ds_edd = ds_edd.resample(time='1M').sum()
            # Append
            ds_edd_out.append(ds_edd) 
            
        except Exception as e:
            except_path = "/storage/work/d/dcl5300/current_projects/wbm_soilM_crop_uc_lafferty-etal-2024-tbd/code/logs/"
            with open(f"{except_path}_{model}__{member}__{ssp}__{str(year)}_.txt", "w") as f:
                f.write(str(e))
            
    # Concat
    ds_edd_out = xr.concat(ds_edd_out, dim="time")
    # Update attrs
    ds_edd_out = ds_edd_out.to_dataset(name = f"edd_{thresh}c")
    ds_edd_out.attrs = ds_tasmin.attrs
    
    ds_edd_out.attrs["NOTE"] = f"Extreme Degree Days (EDD) calculated as in DOI: 10.1111/agec.12315 Supplementary Material with threshold {thresh}C. Author: David Lafferty - University of Illinois (davidcl2@illinois.edu). Date: September 2023"
    ds_edd_out[f"edd_{thresh}c"].attrs["units"] = "Days"
    ds_edd_out[f"edd_{thresh}c"].attrs["description"] = "Extreme (Growing) Degree Days"
        
    # Store
    if ensemble == "OakRidge":
        ds_edd_out.to_netcdf(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}_{method}.nc")
    else:
        ds_edd_out.to_netcdf(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}.nc")

In [26]:
####################################
###### LOCA proj EDD function ######
## (different due to time slicing ##
####################################
def edd_loca_proj(wbm_path, model, member, ssp, thresh, out_path):
    time_ids = ["early", "mid", "late"]
    time_years = [[2015,2044], [2045,2074], [2075,2100]]
    ds_edd_out = []

    # Loop through time slices
    for time_idx in range(3):
        for year in range(time_years[time_idx][0], time_years[time_idx][1]+1):
            # Read
            ds_tasmax = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{ssp}_{time_ids[time_idx]}_tasmax_daily/{year}.nc")
            ds_tasmin = xr.open_dataset(f"{wbm_path}/{ensemble}_{model}_{member}_{ssp}_{time_ids[time_idx]}_tasmin_daily/{year}.nc")
            # Calculate EDD
            ds_edd = edd_ufunc(tasmin = ds_tasmin['tasmin'],
                               tasmax = ds_tasmax['tasmax'],
                               threshold = thresh + 273.15)
            # Sum to monthly
            ds_edd = ds_edd.resample(time='1M').sum()
            # Append
            ds_edd_out.append(ds_edd)
        
    # Concat
    ds_edd_out = xr.concat(ds_edd_out, dim="time")
    # Update attrs
    ds_edd_out = ds_edd_out.to_dataset(name = f"edd_{thresh}")
    ds_edd_out.attrs = ds_tasmin.attrs
    
    ds_edd_out.attrs["NOTE"] = f"Extreme Degree Days (EDD) calculated as in DOI: 10.1111/agec.12315 Supplementary Material with threshold {thresh}C. Author: David Lafferty - University of Illinois (davidcl2@illinois.edu). Date: August 2023"
    ds_edd_out.EDD.attrs["units"] = "Days"
    ds_edd_out.EDD.attrs["description"] = "Extreme (Growing) Degree Days"
    
    # Store
    ds_edd_out.to_netcdf(f"{out_path}/metrics/edd/edd_{thresh}c_LOCA2_{model}_{member}_{ssp}.nc")

### LOCA

In [17]:
###################
# LOCA historical #
###################

# Constants
ensemble = "LOCA2"
ssp = "historical"
vars_ = ["tasmax","tasmin","pr"]

thresh = 29

year_start = 1980
year_end = 2014

# Get run infos
run_info = df_climate_drivers[(df_climate_drivers.ensemble == ensemble) & (df_climate_drivers.ssp == ssp)].copy()

# Do it
delayed = []

for _, info in run_info.iterrows():
    model = info["model"]
    member = info["member"]
    # Check if done
    if not os.path.isfile(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}.nc"): 
        # Calculate EDD
        delayed.append(dask.delayed(edd_spooled)(wbm_path=wbm_path,
                                                 ensemble=ensemble,
                                                 model=model,
                                                 member=member,
                                                 ssp=ssp,
                                                 method="",
                                                 year_start=year_start,
                                                 year_end=year_end,
                                                 thresh=thresh,
                                                 out_path=out_path))
        
# _ = dask.compute(*delayed)

In [18]:
%%time
edd_spooled(wbm_path=wbm_path,
            ensemble="LOCA2",
            model="AWI-CM-1-1-MR",
            member="r5i1p1f1",
            ssp="historical",
            method="",
            year_start=year_start,
            year_end=year_end,
            thresh=thresh,
            out_path=out_path)

CPU times: user 5min 19s, sys: 3min 43s, total: 9min 3s
Wall time: 9min 22s


In [29]:
# Constants
ensemble = "LOCA2"
vars_ = ["tasmax","tasmin","pr"]

thresh = 29

# Get run infos
run_info = df_climate_drivers[(df_climate_drivers.ensemble == ensemble) & (df_climate_drivers.ssp != "historical")].copy()

### NEX

In [15]:
# NEX
model_infos = [{"model": "ACCESS-CM2", "member": "r1i1p1f1"},
               {"model": "BCC-CSM2-MR", "member": "r1i1p1f1"},
               {"model": "CNRM-ESM2-1", "member": "r1i1p1f2"},
               {"model": "MPI-ESM1-2-HR", "member": "r1i1p1f1"},
               {"model": "MRI-ESM2-0", "member": "r1i1p1f1"},
              {"model": "NorESM2-MM", "member": "r1i1p1f1"}]

In [16]:
# NEX-GDDP
ensemble = "NEX-GDDP"
ssp = "historical"

thresh = 29

year_start = 1980
year_end = 2014

delayed = []

for model_info in model_infos:
    model = model_info["model"]
    member = model_info["member"]
    # Check if done
    if not os.path.isfile(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}.nc"): 
        # Calculate EDD
        delayed.append(dask.delayed(edd_spooled)(wbm_path=wbm_path,
                                                 ensemble=ensemble,
                                                 model=model,
                                                 member=member,
                                                 ssp=ssp,
                                                 method="",
                                                 year_start=year_start,
                                                 year_end=year_end,
                                                 thresh=thresh,
                                                 out_path=out_path))
_ = dask.compute(*delayed)

In [10]:
# NEX-GDDP
ensemble = "NEX-GDDP"
ssp = "ssp585"

thresh = 29

year_start = 2015
year_end = 2099

delayed = []

for model_info in model_infos:
    model = model_info["model"]
    member = model_info["member"]
    # Check if done
    if not os.path.isfile(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}.nc"): 
        # Calculate EDD
        delayed.append(dask.delayed(edd_spooled)(wbm_path=wbm_path,
                                                 ensemble=ensemble,
                                                 model=model,
                                                 member=member,
                                                 ssp=ssp,
                                                 method="",
                                                 year_start=year_start,
                                                 year_end=year_end,
                                                 thresh=thresh,
                                                 out_path=out_path))
_ = dask.compute(*delayed)

## OakRidge

In [12]:
methods = ["DBCCA_Daymet", "RegCM_Daymet", "DBCCA_Livneh", "RegCM_Livneh"]

model_infos = [{"model": "ACCESS-CM2", "member": "r1i1p1f1"},
               {"model": "BCC-CSM2-MR", "member": "r1i1p1f1"},
               {"model": "CNRM-ESM2-1", "member": "r1i1p1f2"},
               {"model": "MPI-ESM1-2-HR", "member": "r1i1p1f1"},
               {"model": "MRI-ESM2-0", "member": "r1i1p1f1"},
              {"model": "NorESM2-MM", "member": "r1i1p1f1"},]

In [14]:
# OakRidge
ensemble = "OakRidge"
ssp = "historical"

thresh = 29

year_start = 1980
year_end = 2014

delayed = []

for model_info in model_infos:
    model = model_info["model"]
    for method in methods:
        member = model_info["member"]
        # Check if done
        if not os.path.isfile(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}_{method}.nc"): 
            # Calculate EDD
            delayed.append(dask.delayed(edd_spooled)(wbm_path=wbm_path,
                                                 ensemble=ensemble,
                                                 model=model,
                                                 member=member,
                                                 ssp=ssp,
                                                 method=method,
                                                 year_start=year_start,
                                                 year_end=year_end,
                                                 thresh=thresh,
                                                 out_path=out_path))
            
_ = dask.compute(*delayed)

In [15]:
# OakRidge
ensemble = "OakRidge"
ssp = "ssp585"

thresh = 29

year_start = 2015
year_end = 2059

delayed = []

for model_info in model_infos:
    model = model_info["model"]
    for method in methods:
        member = model_info["member"]
        # Check if done
        if not os.path.isfile(f"{out_path}/metrics/edd/edd_{thresh}c_{ensemble}_{model}_{member}_{ssp}_{method}.nc"): 
            # Calculate EDD
            delayed.append(dask.delayed(edd_spooled)(wbm_path=wbm_path,
                                                 ensemble=ensemble,
                                                 model=model,
                                                 member=member,
                                                 ssp=ssp,
                                                 method=method,
                                                 year_start=year_start,
                                                 year_end=year_end,
                                                 thresh=thresh,
                                                 out_path=out_path))
            
_ = dask.compute(*delayed)